In [8]:
#!/usr/bin/python
# -*- encoding: utf-8 -*-

from models.model import BiSeNet

import torch
import torchvision.transforms as transforms

import numpy as np
from PIL import Image
import cv2
import math
import time
import json
import io
import attr
import os

In [9]:
# Convert the HSV color space to the RGB color space
def hsv2rgb(h, s, v):
    h = float(h)
    s = float(s)
    v = float(v)
    h60 = h / 60.0
    h60f = math.floor(h60)
    hi = int(h60f) % 6
    f = h60 - h60f
    p = v * (1 - s)
    q = v * (1 - f * s)
    t = v * (1 - (1 - f) * s)
    r, g, b = 0, 0, 0
    if hi == 0: r, g, b = v, t, p
    elif hi == 1: r, g, b = q, v, p
    elif hi == 2: r, g, b = p, v, t
    elif hi == 3: r, g, b = p, q, v
    elif hi == 4: r, g, b = t, p, v
    elif hi == 5: r, g, b = v, p, q
    r, g, b = int(r * 255), int(g * 255), int(b * 255)
    return r, g, b


In [10]:
# Convert the RGB color space to the HSV color space
def rgb2hsv(r, g, b):
    r, g, b = r/255.0, g/255.0, b/255.0
    mx = max(r, g, b)
    mn = min(r, g, b)
    m = mx-mn
    if mx == mn:
        h = 0
    elif mx == r:
        if g >= b:
            h = ((g-b)/m)*60
        else:
            h = ((g-b)/m)*60 + 360
    elif mx == g:
        h = ((b-r)/m)*60 + 120
    elif mx == b:
        h = ((r-g)/m)*60 + 240
    if mx == 0:
        s = 0
    else:
        s = m/mx
    v = mx
    # h:0-360, s:0-1, v:0-1
    # H:0-180, S:0-255, V:0-255
    H = h / 2
    S = s * 255.0
    V = v * 255.0
    return H, S, V

In [11]:
# Convert the RGB color space to the HEX color space
def rgb2hex(r, g, b):
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)

In [12]:
# Color distance between two RGB colors
def ColourDistance(rgb_1, rgb_2):
    R_1, G_1, B_1 = rgb_1
    R_2, G_2, B_2 = rgb_2
    rmean = (R_1 + R_2) / 2
    R = R_1 - R_2
    G = G_1 - G_2
    B = B_1 - B_2
    return math.sqrt((2 + rmean / 256) * (R ** 2) + 4 * (G ** 2) + (2 + (255 - rmean) / 256) * (B ** 2))


In [13]:
# RMBD
def rmDarkBright(imgArray, parsing):
    for i in range(512):
        for j in range(512):
            rgb = imgArray[i][j]
            r = rgb[0]
            g = rgb[1]
            b = rgb[2]
            h, s, v = rgb2hsv(r, g, b)
            if (parsing[i][j] == 12 or parsing[i][j] == 13 or parsing[i][j] == 10) and (v < 5 or v > 250):
                parsing[i][j] = 1
    return parsing

In [14]:
# Get ret_dic
with open('./json/lipsticksMod.json', 'r', encoding='utf-8') as f:
# with open('data.json', 'r', encoding='utf-8') as f:
    ret_dic = json.load(f)
    sum = len(ret_dic)
    RGB_array = np.zeros((sum, 3), dtype=int)
    for i in range(sum):
        color_value = ret_dic[i]['color']
        ret_dic[i]['distance'] = 999
        ret_dic[i]['rgb'] = [0, 0, 0]
        ret_dic[i]['rgb'][0] = int(color_value[1:3], 16)
        ret_dic[i]['rgb'][1] = int(color_value[3:5], 16)
        ret_dic[i]['rgb'][2] = int(color_value[5:7], 16)

In [15]:
# Load into memory
# Load model
n_classes = 19
net = BiSeNet(n_classes=n_classes)
save_pth = '../res/cp/79999_iter.pth'
net.load_state_dict(torch.load(save_pth, map_location=torch.device('cpu')))
net.eval()

BiSeNet(
  (cp): ContextPath(
    (resnet): Resnet18(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, a

In [16]:
# Tensor
to_tensor = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [17]:
# Count correct number and fail number
shootCount = 0
failCount = 0

In [11]:
# Test sub-dataset dir
datasetDirPath = "./testDataset/dataset" # Change para

In [21]:
list_lip = []
list_nose = []
list_id = []
id = 0
for img in os.listdir(datasetDirPath):
    with torch.no_grad():
        image = Image.open(datasetDirPath + "/" + img).convert('RGB')
        imgArray = np.array(image)
        image = to_tensor(image)
        image = torch.unsqueeze(image, 0)
        out = net(image)[0]
        parsing = out.squeeze(0).cpu().numpy().argmax(0)
        parsing = rmDarkBright(imgArray, parsing)
        nosePos = np.where(parsing == 10)
        upperLipPos = np.where(parsing == 12)
        lowerLipPos = np.where(parsing == 13)

        pointsCount = len(upperLipPos[0]) + len(lowerLipPos[0])
        pointsCountUpper = len(upperLipPos[0])
        pointsCountLower = len(lowerLipPos[0])
        pointsCountNose = len(nosePos[0])
        rgb = [0, 0, 0]
        rgbn = [0, 0, 0]
        if len(upperLipPos) > 0:
            rgbUpper = np.sum(imgArray[upperLipPos[0], upperLipPos[1], :], axis=0)
            rgb = np.sum([rgb, rgbUpper], axis=0)
        if len(lowerLipPos) > 0:
            rgbLower = np.sum(imgArray[lowerLipPos[0], lowerLipPos[1], :], axis=0)
            rgb = np.sum([rgb, rgbLower], axis=0)
        if len(nosePos) > 0:
            rgbNose = np.sum(imgArray[nosePos[0], nosePos[1], :], axis=0)
            rgbn = np.sum([rgbn, rgbNose], axis=0)
        if pointsCount > 0:
            res = [math.floor(i / pointsCount) for i in rgb]
            resn = [math.floor(i / pointsCountNose) for i in rgbn]
            front = img.split('.')[0]
            imgId = front.split('_')[0]
            list_lip.append(rgb2hex(res[0],res[1],res[2]))
            list_nose.append(rgb2hex(resn[0],resn[1],resn[2]))
            id += 1
            list_id.append(id)
            print('imgId: ' + imgId + ', lip: ' + str(res) + ' nose:' + str(resn))

imgId: face0, lip: [127, 65, 57] nose:[168, 107, 75]
imgId: face10, lip: [193, 125, 116] nose:[220, 167, 137]
imgId: face12, lip: [163, 105, 114] nose:[217, 189, 177]
imgId: face13, lip: [187, 127, 109] nose:[210, 170, 139]
imgId: face14, lip: [151, 85, 62] nose:[217, 172, 140]
imgId: face15, lip: [129, 60, 44] nose:[216, 165, 131]
imgId: face16, lip: [201, 104, 116] nose:[215, 152, 129]
imgId: face17, lip: [151, 95, 76] nose:[212, 173, 146]
imgId: face2, lip: [173, 82, 125] nose:[208, 159, 160]
imgId: face21, lip: [217, 105, 79] nose:[231, 176, 141]
imgId: face22, lip: [189, 102, 82] nose:[214, 142, 103]
imgId: face23, lip: [92, 30, 41] nose:[201, 173, 149]
imgId: face24, lip: [172, 119, 105] nose:[208, 173, 156]
imgId: face25, lip: [90, 22, 37] nose:[201, 171, 148]
imgId: face26, lip: [159, 122, 127] nose:[199, 177, 171]
imgId: face27, lip: [159, 126, 130] nose:[188, 166, 160]
imgId: face28, lip: [143, 103, 100] nose:[191, 173, 168]
imgId: face4, lip: [85, 48, 51] nose:[202, 176, 164

In [22]:
print(list_lip)
print(list_nose)
print(list_id)

['#7f4139', '#c17d74', '#a36972', '#bb7f6d', '#97553e', '#813c2c', '#c96874', '#975f4c', '#ad527d', '#d9694f', '#bd6652', '#5c1e29', '#ac7769', '#5a1625', '#9f7a7f', '#9f7e82', '#8f6764', '#553033', '#a25754', '#b77088', '#b68069']
['#a86b4b', '#dca789', '#d9bdb1', '#d2aa8b', '#d9ac8c', '#d8a583', '#d79881', '#d4ad92', '#d09fa0', '#e7b08d', '#d68e67', '#c9ad95', '#d0ad9c', '#c9ab94', '#c7b1ab', '#bca6a0', '#bfada8', '#cab0a4', '#b68d7e', '#ddada5', '#cb9b7a']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


In [25]:
listofjson = [{'lip': lip, 'color': color, 'id': id} for lip, color, id in zip(list_lip, list_nose, list_id)]
listofjson

[{'lip': '#7f4139', 'color': '#a86b4b', 'id': 1},
 {'lip': '#c17d74', 'color': '#dca789', 'id': 2},
 {'lip': '#a36972', 'color': '#d9bdb1', 'id': 3},
 {'lip': '#bb7f6d', 'color': '#d2aa8b', 'id': 4},
 {'lip': '#97553e', 'color': '#d9ac8c', 'id': 5},
 {'lip': '#813c2c', 'color': '#d8a583', 'id': 6},
 {'lip': '#c96874', 'color': '#d79881', 'id': 7},
 {'lip': '#975f4c', 'color': '#d4ad92', 'id': 8},
 {'lip': '#ad527d', 'color': '#d09fa0', 'id': 9},
 {'lip': '#d9694f', 'color': '#e7b08d', 'id': 10},
 {'lip': '#bd6652', 'color': '#d68e67', 'id': 11},
 {'lip': '#5c1e29', 'color': '#c9ad95', 'id': 12},
 {'lip': '#ac7769', 'color': '#d0ad9c', 'id': 13},
 {'lip': '#5a1625', 'color': '#c9ab94', 'id': 14},
 {'lip': '#9f7a7f', 'color': '#c7b1ab', 'id': 15},
 {'lip': '#9f7e82', 'color': '#bca6a0', 'id': 16},
 {'lip': '#8f6764', 'color': '#bfada8', 'id': 17},
 {'lip': '#553033', 'color': '#cab0a4', 'id': 18},
 {'lip': '#a25754', 'color': '#b68d7e', 'id': 19},
 {'lip': '#b77088', 'color': '#ddada5', 

In [37]:
# Crop 512x512
list_test = []
list_title = []
testDirPath = "./testDataset/test/" # Change para
num = 1
for img in os.listdir(testDirPath):
    image = Image.open(testDirPath+img)
    print(image)
    # front = image.split('.')[0]
    # imgId = front.split('_')[0]
    list_test.append(image.resize((512, 512)))
    list_title.append(num)
    num += 1

list_dict = [{'img id': title, 'img': test} for title, test in zip(list_title, list_test)]

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1479x1109 at 0x189F9ED5930>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1108x1478 at 0x189F9377790>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1108x1478 at 0x189F9ED5930>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=736x984 at 0x189F9377CA0>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=750x750 at 0x189F9376260>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=236x314 at 0x189F93771F0>


In [35]:
for dict in list_dict:
    with torch.no_grad():
        # image = Image.open(img).convert('RGB')
        image = dict['img'].convert('RGB')
        imgArray = np.array(image)
        image = to_tensor(image)
        image = torch.unsqueeze(image, 0)
        out = net(image)[0]
        parsing = out.squeeze(0).cpu().numpy().argmax(0)
        parsing = rmDarkBright(imgArray, parsing)
        nosePos = np.where(parsing == 10)

        pointsCountNose = len(nosePos[0])

        rgbn = [0, 0, 0]
        if len(nosePos) > 0:
            rgbNose = np.sum(imgArray[nosePos[0], nosePos[1], :], axis=0)
            rgbn = np.sum([rgbn, rgbNose], axis=0)
        if pointsCountNose > 0:
            res = [math.floor(i / pointsCountNose) for i in rgbn]
            for i in range(sum):
                ret_dic[i]['distance'] = ColourDistance(res, ret_dic[i]['rgb'])
            predictTmp = sorted(ret_dic, key=lambda x: float(x['distance']))[:5]
            predictRes = []
            for lipstick in predictTmp:
                predictRes.append(lipstick['lip'])

            imgId = dict['img id']
            print('imgId: ' + str(imgId) + ', nose:' + str(res) + ', predictRes: ' + str(predictRes))

            print(rgb2hex(res[0],res[1],res[2]))
#             if (imgId) in predictRes:
#                 shootCount += 1
#             else:
#                 failCount += 1
# print(testDirPath + ': ' + str(shootCount) + ' success, ' + str(failCount) + ' fails, rate: ' + str(shootCount/(shootCount+failCount)))

imgId: 1, nose:[187, 143, 123], predictRes: ['#854F3C', '#C06B63', '#8C5859', '#8A5145', '#B26158']
#bb8f7b
imgId: 2, nose:[169, 133, 103], predictRes: ['#944E3B', '#996656', '#915240', '#8E2224', '#A25B4B']
#a98567
imgId: 3, nose:[180, 153, 123], predictRes: ['#854F3C', '#622F2C', '#83473A', '#601C24', '#A55354']
#b4997b
imgId: 4, nose:[138, 99, 71], predictRes: ['#83242C', '#562C37', '#6C3841', '#915240', '#996656']
#8a6347
imgId: 5, nose:[110, 86, 74], predictRes: ['#83242C', '#562C37', '#6C3841', '#7F394D', '#7A4350']
#6e564a
imgId: 6, nose:[220, 197, 188], predictRes: ['#955B55', '#BD8881', '#C9826C', '#B07C79', '#B18286']
#dcc5bc


In [32]:
for img in os.listdir(testDirPath):
    with torch.no_grad():
        image = Image.open(testDirPath + "/" + img).convert('RGB')
        imgArray = np.array(image)
        imgArray2 = np.array(image)
        image = to_tensor(image)
        image = torch.unsqueeze(image, 0)
        out = net(image)[0]
        parsing = out.squeeze(0).cpu().numpy().argmax(0)
        parsing = rmDarkBright(imgArray, parsing)
        nosePos = np.where(parsing == 10)
        upperLipPos = np.where(parsing == 12)
        lowerLipPos = np.where(parsing == 13)

        pointsCount = len(upperLipPos[0]) + len(lowerLipPos[0])
        pointsCountUpper = len(upperLipPos[0])
        pointsCountLower = len(lowerLipPos[0])
        pointsCountNose = len(nosePos[0])
        rgb = [0, 0, 0]
        rgbn = [0, 0, 0]
        if len(upperLipPos) > 0:
            rgbUpper = np.sum(imgArray[upperLipPos[0], upperLipPos[1], :], axis=0)
            rgb = np.sum([rgb, rgbUpper], axis=0)
        if len(lowerLipPos) > 0:
            rgbLower = np.sum(imgArray[lowerLipPos[0], lowerLipPos[1], :], axis=0)
            rgb = np.sum([rgb, rgbLower], axis=0)
        if len(nosePos) > 0:
            rgbNose = np.sum(imgArray2[nosePos[0], nosePos[1], :], axis=0)
            rgbn = np.sum([rgbn, rgbNose], axis=0)
        if pointsCount > 0:
            res = [math.floor(i / pointsCount) for i in rgb]
            ress = [math.floor(i / pointsCountNose) for i in rgbn]
            for i in range(sum):
                ret_dic[i]['distance'] = ColourDistance(ress, ret_dic[i]['rgb'])
            predictTmp = sorted(ret_dic, key=lambda x: float(x['distance']))[:5]
            predictRes = []
            for lipstick in predictTmp:
                predictRes.append(lipstick['lip'])
            front = img.split('.')[0]
            imgId = front.split('_')[0]
            print('imgId: ' + imgId + ', lip: ' + str(res) + ' nose:' + str(ress) + ', predictRes: ' + str(predictRes))
            if (imgId) in predictRes:
                shootCount += 1
            else:
                failCount += 1
# print(testDirPath + ': ' + str(shootCount) + ' success, ' + str(failCount) + ' fails, rate: ' + str(shootCount/(shootCount+failCount)))

imgId: 1, lip: [184, 108, 109] nose:[190, 145, 124], predictRes: ['#854F3C', '#C06B63', '#B26158', '#A86863', '#A56354']
imgId: 2, lip: [155, 107, 99] nose:[174, 139, 107], predictRes: ['#944E3B', '#6C0F19', '#8E2224', '#8F4945', '#8A5145']
imgId: 3, lip: [142, 104, 98] nose:[172, 144, 116], predictRes: ['#622F2C', '#8F4945', '#8A5145', '#944E3B', '#6C0F19']
imgId: 4, lip: [101, 61, 53] nose:[147, 105, 75], predictRes: ['#83242C', '#915240', '#6C3841', '#996656', '#562C37']
imgId: 5, lip: [86, 67, 63] nose:[110, 86, 74], predictRes: ['#83242C', '#562C37', '#6C3841', '#7F394D', '#7A4350']
imgId: 6, lip: [212, 160, 150] nose:[221, 199, 190], predictRes: ['#AC7770', '#80404B', '#B07C79', '#BD8881', '#B07F7A']
